In [1]:
!pip install sktime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 5.2 MB/s 


In [123]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.datasets import load_basic_motions  # multivariate dataset
from sktime.transformations.panel.rocket import MiniRocketMultivariate

from sklearn.model_selection import train_test_split

In [124]:
import pandas as pd


def str2listf(l):
    l = l.replace("[", "")
    l = l.replace("]", "")
    l = l.replace("\n", "")
    l = l.split(', ')
    l = [float(i) for i in l]
    return l


In [141]:
df = pd.read_csv('data_standard.csv')
df.head()

,Unnamed: 0,participant_name,pattern_name,ts_distance,ts_pupil,x_coord,y_coord,speed,series_original_lenght
0,0,Anoth,pattern3,"[0.023857761366087755, 0.023857761366087755, 0...","[2.2533971438073968, 2.2533971438073968, 2.253...","[0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.2...","[0.136, 0.136, 0.136, 0.136, 0.136, 0.136, 0.1...",2.308106,67
1,1,Anoth,pattern6,"[-0.5749046258955413, -0.5804129627747906, -0....","[1.5225453984827109, 1.5180912868044785, 1.512...","[0.277, 0.277, 0.277, 0.277, 0.277, 0.2776, 0....","[0.138, 0.1389, 0.1398, 0.1406, 0.1415, 0.1426...",2.178072,71
2,2,Anoth,pattern1,"[0.2661898324679135, 0.18440249844628362, 0.10...","[-0.6410080528762702, -0.5282003930303925, -0....","[0.278, 0.2783, 0.2787, 0.279, 0.2794, 0.2797,...","[0.119, 0.1197, 0.1204, 0.1211, 0.1221, 0.1232...",2.761485,56
3,3,Anoth,pattern3,"[-0.2055720954953975, -0.18520777777519518, -0...","[0.6027182572572641, 0.6248040563814077, 0.648...","[0.275, 0.2754, 0.2758, 0.2758, 0.2754, 0.275,...","[0.129, 0.129, 0.129, 0.1288, 0.1283, 0.128, 0...",2.308106,67
4,4,Anoth,pattern4,"[-0.8932606811727897, -0.8490280679877931, -0....","[1.2782153861134196, 1.2924982126010405, 1.306...","[0.29, 0.2887, 0.2873, 0.286, 0.2857, 0.2853, ...","[0.145, 0.1447, 0.1443, 0.144, 0.143, 0.142, 0...",2.863762,54


In [142]:
# change datatypes

# del unnamed and make it the index
df.index = df['Unnamed: 0'].tolist()
del df['Unnamed: 0']

# cast values from string to float
df.ts_distance = df.ts_distance.apply(str2listf)
df.ts_pupil = df.ts_pupil.apply(str2listf)
df.x_coord = df.x_coord.apply(str2listf)
df.y_coord = df.y_coord.apply(str2listf)
df['series_original_lenght'] = df['series_original_lenght'].apply(lambda i: int(i))
df['speed'] = df['speed'].apply(lambda i: float(i))
df

,participant_name,pattern_name,ts_distance,ts_pupil,x_coord,y_coord,speed,series_original_lenght
0,Anoth,pattern3,"[0.023857761366087755, 0.023857761366087755, 0...","[2.2533971438073968, 2.2533971438073968, 2.253...","[0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.2...","[0.136, 0.136, 0.136, 0.136, 0.136, 0.136, 0.1...",2.308106,67
1,Anoth,pattern6,"[-0.5749046258955413, -0.5804129627747906, -0....","[1.5225453984827109, 1.5180912868044785, 1.512...","[0.277, 0.277, 0.277, 0.277, 0.277, 0.2776, 0....","[0.138, 0.1389, 0.1398, 0.1406, 0.1415, 0.1426...",2.178072,71
2,Anoth,pattern1,"[0.2661898324679135, 0.18440249844628362, 0.10...","[-0.6410080528762702, -0.5282003930303925, -0....","[0.278, 0.2783, 0.2787, 0.279, 0.2794, 0.2797,...","[0.119, 0.1197, 0.1204, 0.1211, 0.1221, 0.1232...",2.761485,56
3,Anoth,pattern3,"[-0.2055720954953975, -0.18520777777519518, -0...","[0.6027182572572641, 0.6248040563814077, 0.648...","[0.275, 0.2754, 0.2758, 0.2758, 0.2754, 0.275,...","[0.129, 0.129, 0.129, 0.1288, 0.1283, 0.128, 0...",2.308106,67
4,Anoth,pattern4,"[-0.8932606811727897, -0.8490280679877931, -0....","[1.2782153861134196, 1.2924982126010405, 1.306...","[0.29, 0.2887, 0.2873, 0.286, 0.2857, 0.2853, ...","[0.145, 0.1447, 0.1443, 0.144, 0.143, 0.142, 0...",2.863762,54
...,...,...,...,...,...,...,...,...
625,Sebastian,pattern5,"[-0.8909659220977034, -0.9389699089318274, -0....","[-1.524967793898241, -1.5410838341238862, -1.5...","[0.26, 0.2605, 0.261, 0.261, 0.261, 0.2618, 0....","[0.133, 0.1344, 0.1358, 0.1364, 0.1369, 0.1374...",1.540069,77
626,Sebastian,pattern1,"[-1.1196175894114557, -1.0865446705877915, -1....","[-1.1744260100172619, -1.1130997888831757, -1....","[0.262, 0.2617, 0.2613, 0.261, 0.2614, 0.2617,...","[0.143, 0.1427, 0.1423, 0.142, 0.1424, 0.1427,...",2.117595,56
627,Sebastian,pattern6,"[-0.7651815905173864, -0.7651815905173864, -0....","[-1.5412592687132425, -1.4184863145815716, -1....","[0.265, 0.2658, 0.2666, 0.267, 0.267, 0.2669, ...","[0.145, 0.145, 0.145, 0.1455, 0.1463, 0.1469, ...",1.769930,67
628,Sebastian,pattern3,"[-0.7627326178949756, -0.8013246830995923, -0....","[0.22035178990155088, 0.23297243670843854, 0.2...","[0.268, 0.2675, 0.267, 0.2665, 0.266, 0.268, 0...","[0.144, 0.1445, 0.145, 0.144, 0.143, 0.143, 0....",1.464016,81


In [143]:
del df['pattern_name']
del df['speed']
del df['series_original_lenght']

In [144]:
# split df in train and X_test 20%

train, test = train_test_split(df, test_size=0.2)

In [145]:
y_train = train['participant_name']
del train['participant_name']
X_train = train
X_train

y_test = test['participant_name']
del test['participant_name']
X_test = test
X_test

,ts_distance,ts_pupil,x_coord,y_coord
367,"[-1.0461930399025214, -1.0281615045700654, -1....","[-0.6187552330607454, -0.6518855923917594, -0....","[0.266, 0.2664, 0.2669, 0.267, 0.267, 0.2668, ...","[0.17, 0.1713, 0.1726, 0.1727, 0.1722, 0.1716,..."
63,"[-0.5093613294627763, -0.5093613294627763, -0....","[1.0583746353270262, 1.0848171210864395, 1.109...","[0.275, 0.275, 0.275, 0.2753, 0.2758, 0.2764, ...","[0.122, 0.122, 0.122, 0.1217, 0.1212, 0.1204, ..."
8,"[-0.627900072201655, -0.6246563496239097, -0.6...","[2.5006976336028917, 2.52552806145712, 2.55035...","[0.273, 0.2735, 0.274, 0.2736, 0.2731, 0.2715,...","[0.129, 0.1285, 0.128, 0.1272, 0.1262, 0.1268,..."
396,"[-0.8626064490906868, -0.758714090403725, -0.6...","[-1.285963184551122, -1.3041186222547463, -1.3...","[0.271, 0.2702, 0.2694, 0.2688, 0.2684, 0.2681...","[0.144, 0.1424, 0.1407, 0.1412, 0.1432, 0.1451..."
250,"[-0.30090287911240166, -0.30090287911240166, -...","[-0.47979853533878525, -0.40319444550386946, -...","[0.279, 0.279, 0.279, 0.279, 0.2797, 0.2803, 0...","[0.132, 0.1317, 0.1313, 0.131, 0.1313, 0.1317,..."
...,...,...,...,...
463,"[0.7886430530433437, 0.7443756598378527, 0.693...","[-1.7907724481353149, -1.8132250892185444, -1....","[0.284, 0.2837, 0.2835, 0.2832, 0.283, 0.283, ...","[0.108, 0.1087, 0.1095, 0.1103, 0.111, 0.1115,..."
334,"[-0.4841163677730231, -0.47938929147359327, -0...","[1.3213507651344223, 1.3750990186167398, 1.376...","[0.271, 0.2703, 0.2696, 0.269, 0.269, 0.2695, ...","[0.157, 0.1577, 0.1572, 0.1565, 0.1593, 0.1595..."
592,"[-0.824195881956706, -0.8279663484929041, -0.8...","[-1.0086129838073103, -1.0293820310921653, -1....","[0.27, 0.27, 0.27, 0.2694, 0.2691, 0.2696, 0.2...","[0.138, 0.1385, 0.139, 0.1384, 0.1379, 0.1368,..."
431,"[-0.6300066929543743, -0.6670000196252306, -0....","[-1.9966559818098002, -1.9288408553031098, -1....","[0.277, 0.2762, 0.277, 0.2777, 0.277, 0.2762, ...","[0.201, 0.2002, 0.1959, 0.192, 0.192, 0.1928, ..."


In [146]:
X_train = X_train.applymap(lambda s: pd.Series(s))
X_test = X_test.applymap(lambda s: pd.Series(s))

#### TRANSFORMATIONS DETAILS: basic_motions dataset

In [ ]:
X_train, y_train = load_basic_motions(split="train", return_X_y=True)
dumm = pd.get_dummies(y_train)
dumm

In [ ]:
dumm = dumm.applymap(lambda s: [s for _ in range(len(X_train.dim_0[0]))])
dumm

In [ ]:
dumm = dumm.applymap(lambda s: pd.Series(s))
dumm

In [ ]:
print(type(X_train.dim_0[0]))
print(type(X_train.badminton[0]))

In [ ]:
X_train = pd.concat([X_train, dumm], 1)
X_train

#### TRAIN

In [147]:
minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(X_train)
X_train_transform = minirocket_multi.transform(X_train)

In [148]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

In [138]:
# del X_test['participant_name']

In [149]:
# X_test, y_test = load_basic_motions(split="test", return_X_y=True)
X_test_transform = minirocket_multi.transform(X_test)

In [160]:
classifier.score(X_test_transform, y_test)

0.9193548387096774

In [152]:
predictions = classifier.predict(X_test_transform)

In [158]:
df_eval = pd.DataFrame([(p, t) for p, t in zip(predictions, y_test.tolist())])

In [159]:
df_eval

,0,1
0,Josephin,Josephin
1,Anoth,Anoth
2,Anoth,Anoth
3,Josephin,Josephin
4,Ashok,Ashok
...,...,...
119,Ashok,Raghu
120,Gowthom,Gowthom
121,Sebastian,Sebastian
122,Josephin,Josephin


In [ ]:
# interesting is that if pass the y dummies as features using series repeating the binary value, it fails to recognize it, being this the only information it would actually need.
# perhaps is no suitable to that kind of features, you better test it without it.